**Implementation of Farquhar Photosynthesis Model in Python

# This is a Jupyter Test:

In [1]:
def squaring(itera, number):
    square = number
    for i in range (itera):
        square = square * square
    return (square)

for i in range(2):
    print_out = squaring(3, 12)
    print(print_out)

429981696
429981696


# Reference implementation by Colorado State University of the Model:

farquhar <- function(V.max=50, J.max=100, APAR=500, c.i=30, temp.C=25) {

Model inputs:
  V.max = maximum rubisco-limited rate in micromoles per (m^2 sec)
  J.max = maximum light-limited rate in micromoles per (m^2 sec)
  APAR = absorbed photosynthetically-active radiation in micromoles per (m^2 sec)
  c.i = intercellular CO2 partial pressure in Pascals (roughly ppm/10)
  temp.C = leaf temperature (Celsius)

Return value = net carbon assimilation (micromoles CO2 per m^2 of leaf area per second)

Some local parameters we need (adjusted for temperature according to Collatz 91)
p.sfc <- 101325  # surface air pressure (Pascals)
O.i <- 0.209 * p.sfc  # oxygen partial pressure in chloroplast

# Adjust some parameters for temperature according to Collatz et al (1991) Table A1
tau <- 2600 * 0.57^((temp.C - 25)/10) # CO2/O2 specificity ratio for rubisco
gamma <- O.i / (2 *tau) # CO2 compensation point (Pascals, Collatz A3)
K.c <- 30 * 2.1^((temp.C - 25)/10)  # Michaelis constant for carboxylation (Pascals)
K.o <- 30000 * 1.2^((temp.C - 25)/10)  # Michaelis constant for oxidation (Pascals)

# Temp-adjusted maximum carboxylation rate
cold <- 10.
hot <- 40.
slope.cold <- .25
slope.heat <- .4
cold.inhibition <- 1 + exp(slope.cold*(cold - temp.C))
heat.inhibition <- 1 + exp(slope.heat*(temp.C - hot))
V.m <- V.max * 2.1 ^ ((temp.C-25)/10) /
       (cold.inhibition * heat.inhibition)

# Temp-adjusted leaf respiration
R.d <- 0.015 * V.m * 2.4 ^ ((temp.C-25)/10) / 
       (1. + exp((1.3*(temp.C - 55))))

# Solution of quadratic (Bonan 17.8)
a <- 0.7
b <- -(J.max + 0.385 * APAR)
c <- 0.385 * J.max * APAR
J.1 <- (-b + sqrt(b^2 - 4 * a * c) ) / (2 * a)
J.2 <- (-b - sqrt(b^2 - 4 * a * c) ) / (2 * a)
J <- min(J.1, J.2) 

# Rubisco-limited rate of photosynthesis
w.c <- V.m * (c.i - gamma) / (c.i + K.c * (1 + O.i/K.o))  # Collatz A5 or Bonan 17.6

# Light-limited rate of photosynthesis
w.j <- J * (c.i - gamma) / (4 * (c.i + 2 * gamma)) # Bonan 17.7 or Collatz A2

# Sink-limited rate of photosynthesis     # Collatz A7
w.s <- V.m / 2

# Net assimilation 
A.n <- min(w.c, w.j, w.s)-R.d

return(A.n)
}

Creation of a function to determine leaf respiration

In [36]:
import math
def leaf_respiration(v_max = 50, leaf_temp = 25):
    step_1 = 0.015 * (v_max * 2.4)
    step_2 = (step_1) ** ((leaf_temp - 25)/10) / (1 + math.exp((1.3*(leaf_temp - 55))))
    return(step_2)

In [37]:
leaf_respiration(50, 27)

1.1247461131420946

# Definition of functions to keep heat and cold inhibition in mind

In [5]:
import math
def cold_inhibition(leaf_temp = 25):
    cold_limit = 40
    cold_slope = 0.4
    c_inhibition = 1 + math.exp(cold_slope*(cold_limit - leaf_temp))
    return(c_inhibition)

In [6]:
cold_inhibition(25)

404.4287934927351

# Definition of function to calculate heat inhibition

In [7]:
import math
def heat_inhibition(leaf_temp = 25):
    heat_limit = 10
    heat_slope = 0.25
    h_inhibition = 1 + math.exp(heat_slope*(heat_limit - leaf_temp))
    return(h_inhibition)

In [8]:
heat_inhibition(30)

1.0067379469990854

# Definition of function to calculate potential electron tranpsort during photosynthesis

In [38]:
import math
def pot_electron_transport(j_max = 100, apar = 500):
    a = 0.7
    b = (j_max + 0.385 * apar)
    c = (0.385 * j_max * apar)
    j_1 = (-b + math.sqrt((b**2) - 4 * a * c) ) / (2 * a)
    j_2 = (-b - math.sqrt((b**2) - 4 * a * c) ) / (2 * a)
    j_correct = min(j_1, j_2)
    return (j_correct)

In [39]:
pot_electron_transport(100, 550)

-361.73160510087

# Definition of the Farquar Photosynthesis Modell Function in Python:

In [40]:
def farquar(v_max = 50, j_max = 100, apar = 500, c_pressure = 30, leaf_temp = 25, surface_pressure = 101325):
    # Some local parameters we need (adjusted for temperature according to Collatz 91)
    partial_oxy_pressure = 0.209 * surface_pressure  # oxygen partial pressure in chloroplast
    
    # Adjust some parameters for temperature according to Collatz et al (1991) Table A1
    tau = 2600 * (0.57**((leaf_temp - 25)/10)) # CO2/O2 specificity ratio for rubisco
    gamma = partial_oxy_pressure / (2 *tau) # CO2 compensation point (Pascals, Collatz A3)
    michaelis_con_carbox = 30 * (2.1**((leaf_temp - 25)/10))  # Michaelis constant for carboxylation (Pascals)
    michaelis_con_oxy = 30000 * (1.2**((leaf_temp - 25)/10))  # Michaelis constant for oxidation (Pascals)

    # Solution of quadratic potential electron transport (Bonan 17.8)
    pot_electron_transport(j_max, apar)
    
    # Solution of maximum rate of carboxylation
    v_max_carbox = v_max * (2.1 ** ((leaf_temp - 25)/10) / (cold_inhibition(leaf_temp) * heat_inhibition(leaf_temp)))

    # Rubisco-limited rate of photosynthesis
    rubi_limited_w = v_max_carbox * ((c_pressure - gamma) / (c_pressure + michaelis_con_carbox * (1 + partial_oxy_pressure/michaelis_con_oxy)))  # Collatz A5 or Bonan 17.6

    # Light-limited rate of photosynthesis
    light_limited_w = pot_electron_transport(j_max, apar) * ((c_pressure - gamma) / (4 * (c_pressure + 2 * gamma))) # Bonan 17.7 or Collatz A2

    # Sink-limited rate of photosynthesis     # Collatz A7
    sink_limited_w = v_max_carbox / 2

    # Net assimilation 
    net_assim = (min(rubi_limited_w, light_limited_w, sink_limited_w)) - leaf_respiration(leaf_temp)
    
    return(net_assim)

In [42]:
farquar(50, 100, 500, 30, 25, 101325)

-58.098305721256125